In [1]:
import pandas as pd
import numpy as np
import os
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from pprint import pprint

**Dataset configuration**

In [2]:
dataset = 'scott-final-iter1'

dataset_path = '../datasets/' + dataset + '/'
output_path = '../datasets/' + dataset + '-fft/'
features_output_path = '../datasets/' + dataset + '-features/'

In [3]:
devices_to_use = [
    '128.237.246.127',
    '128.237.248.186',
    '128.237.247.134',
    '128.237.234.0',
    '128.237.237.122',
    '128.237.239.234',
    '128.237.254.195', # this Mite is only in the Synergy kitchen dataset
#     'DialogIoT 591844595',
#     'DialogIoT 591844599',
#     'DialogIoT 591844765',
#     'Matrix b827eb96f31a',
#     'Matrix b827ebe6e0f8',
#     'Matrix b827eb41f96f',
#     'TI SensorTag 604',
#     'TI SensorTag 690',
#     'TI SensorTag 85',
#     'xdk_1',
#     'xdk_2',
#     'xdk_3',
#     'TI SensorTag 33',
#     'TI SensorTag 535',
#     'TI SensorTag 709'
]

columns_to_rename = {
    'ACCEL_sst_0_avg': 'accel_x',
    'ACCEL_sst_1_avg': 'accel_y',
    'ACCEL_sst_2_avg': 'accel_z',
    'MAGNETOMETER_sst_0_avg': 'mag_x',
    'MAGNETOMETER_sst_1_avg': 'mag_y',
    'MAGNETOMETER_sst_2_avg': 'mag_z',
    'HUMIDITY_sst_0_avg': 'humidity',
    'ILLUMINATION_sst_0_avg': 'light',
    'BAROMETER_sst_0_avg': 'pressure',
    'TEMPERATURE_sst_0_avg': 'temperature',
    'MICROPHONE_sst_0_avg': 'microphone',
    'microphone_avg': 'microphone',
    'magnetometer_x': 'mag_x',
    'magnetometer_y': 'mag_y',
    'magnetometer_z': 'mag_z'
}

columns_to_keep = '|'.join([
#     'accel_',
#     'gyro_',
#     'mag_',
#     'humidity',
#     'light',
#     'pressure',
#     'temperature',
#     'microphone',
    'ACCEL_fft_',
    'MICROPHONE_fft_',
])

**Merge the individual CSVs into one DataFrame per device for the chosen devices**

In [4]:
data_frames = {}

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        device_id = None

        for device in devices_to_use:
            if file.startswith(device):
                device_id = device

        if device_id is not None:
            df = pd.DataFrame()
            if device_id in data_frames:
                df = data_frames[device_id]
            
            df_new = pd.DataFrame.from_csv(dataset_path + file)
            df_new.rename(index=str, columns=columns_to_rename, inplace=True)

            df_new = df_new.filter(regex=(columns_to_keep))

            if len(df_new.columns) > 0:
                df = df.join(df_new, how='outer')

                data_frames[device_id] = df

**Remove duplicate indices**

In [5]:
for device_id in data_frames:
    df = data_frames[device_id]
    df = df[~df.index.duplicated(keep='first')]
    data_frames[device_id] = df

**Convert index to datetime**

In [6]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.index = pd.to_datetime(df.index)
    data_frames[device_id] = df

**Fill missing values**

In [7]:
for device_id in data_frames:
    df = data_frames[device_id]
    df = df.ffill().bfill()
    data_frames[device_id] = df

**Create 5 sec activity windows**

In [8]:
window_size_secs = 5 # seconds
smallest_size_secs = 3 # seconds

activities = pd.DataFrame.from_csv(dataset_path + 'activities.csv')
labels = pd.DataFrame.from_csv(dataset_path + 'activity_labels.csv')
labeled_activities = activities.loc[activities.id != -1]

activity_windows = []

for id in labeled_activities.id.unique():
    start = labeled_activities.loc[labeled_activities.id == id].index.min()
    end = labeled_activities.loc[labeled_activities.id == id].index.max()

    since = start
    until = start + pd.DateOffset(seconds=window_size_secs)

    while since < end:
        label = labels.loc[id]['label']

        length = (until - since).seconds
        if length >= smallest_size_secs:
            activity_windows.append({
                'since': since,
                'until': until,
                'label': label,
                'length': length,
                'id': id
            })

        since = until
        until = until + pd.DateOffset(seconds=window_size_secs)
        until = min(until, end)

**Create per-activity dataframes which are not missing more than 1 second of data**

In [9]:
activity_data_frames = {}
device_labels = {}

removed_activities = {}
all_activities = {}
activity_id = 0

for device_id in data_frames:
    df = data_frames[device_id]
    data_frames[device_id] = None
    df['id'] = -1

    device_activities = []

    for i, window in enumerate(activity_windows):

        activity_df = df.loc[df.index >= window['since']]
        activity_df = activity_df.loc[activity_df.index <= window['until']].copy()

        seconds = activity_df.index.round('s').unique()

        key = device_id + ', activity ' + str(window['label'])
        if not key in all_activities:
            all_activities[key] = 0
            removed_activities[key] = 0

        if len(seconds) + 1 < window['length']:
            removed_activities[key] += 1
        else:
            activity_df['id'] = activity_id
            activity_data_frames[activity_id] = activity_df
            device_activities.append(pd.DataFrame({
                'id': [activity_id],
                'label': [window['label']],
                'activity_id': [window['id']],
                'since': [window['since']],
                'until': [window['until']],
                'window_id': [i]
            }))
            activity_id += 1

        all_activities[key] += 1

    df_labels = pd.concat(device_activities)
    df_labels = df_labels.set_index('id')
    device_labels[device_id] = df_labels

# print stats    
for key in all_activities:
    if removed_activities[key] > 0:
        print(str(removed_activities[key]) + ' out of ' + str(all_activities[key]) + ' removed for ' + key)

2 out of 61 removed for 128.237.247.134, activity 1


**Resample to 10Hz and fill missing values**

In [10]:
for activity_id in activity_data_frames:
    df = activity_data_frames[activity_id]
    df = df.resample('100L')
    df = df.ffill().bfill()
    activity_data_frames[activity_id] = df

**Update device data frames**

In [11]:
for device_id in device_labels:
    labels = device_labels[device_id]
    df_parts = [activity_data_frames[id] for id in labels.index]
    df = pd.concat(df_parts)
    data_frames[device_id] = df

**Replace timestamp indices with integer ones**

In [12]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.insert(0, 'i_index', pd.Series(range(len(df.index)), index=df.index))
    df = df.set_index(['i_index'])
    data_frames[device_id] = df

**Save as Pickle files to disk**

In [13]:
for device_id in data_frames:
    df = data_frames[device_id]
    df.to_pickle(output_path + device_id + '.p')

    df_labels = device_labels[device_id]
    df_labels.to_pickle(output_path + device_id + '_labels.p')

**Extract features**

In [14]:
data_frame_features = {}

def extract_device_features(df):
    extracted_features = extract_features(df, column_id="id")
    impute(extracted_features)
    return extracted_features

for device_id in data_frames:
    df = data_frames[device_id]

    extracted = extract_device_features(df)
    data_frame_features[device_id] = extracted

Feature Extraction: 100%|██████████| 10/10 [02:40<00:00, 11.97s/it]
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_3'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_2'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_1'
 'mag_y__friedrich_coefficients__m_3__r_30__coeff_0'
 'accel_x__max_langevin_fixed_point__m_3__r_30'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_3'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_2'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_1'
 'accel_x__friedrich_coefficients__m_3__r_30__coeff_0'
 'gyro_x__max_langevin_fixed_point__m_3__r_30'
 'gyro_x__friedrich_coefficients__m_3__r_30__coeff_3'
 'gyro_x__friedrich_coefficients__m_3__r_30__coeff_2'
 'gyro_x__friedrich_coefficients__m_3__r_30__coeff_1'
 'gyro_x__friedrich_coefficients__m_3__r_30__coeff_0'
 'gyro_z__max_langevin_fixed_point__m_3__r_30'
 'gyro_z__friedrich_coefficients__m_3__r_30__coeff_3'
 'gyro_z__friedrich_coefficients__m_3__r_30__coeff_2'
 'gyro_z__friedrich_coefficients__

**Sort feature columns**

In [15]:
for device_id in data_frame_features:
    df = data_frame_features[device_id]
    df = df.reindex_axis(sorted(df.columns), axis=1)
    data_frame_features[device_id] = df

**Store extracted features on disk**

In [16]:
for device_id in data_frame_features:
    extracted = data_frame_features[device_id]
    extracted.to_pickle(features_output_path + device_id + '.p')

    df_labels = device_labels[device_id]
    df_labels.to_pickle(features_output_path + device_id + '_labels.p')

**Reload extracted datasets from disk**

In [29]:
data_frame_features = {}
device_labels = {}

for root, dirs, files in os.walk(features_output_path):
    for file in files:
        device_id = None

        for device in devices_to_use:
            if file.startswith(device):
                device_id = device

        if device_id is not None:
            df = pd.read_pickle(features_output_path + file)
            if file.endswith('_labels.p'):
                device_labels[device_id] = df
            else:
                data_frame_features[device_id] = df

**Do feature selection**

In [31]:
data_frame_selected = {}

for device_id in device_labels:
    df = data_frame_features[device_id]
    df_labels = device_labels[device_id]
    selected = select_features(df, df_labels['label'])
    data_frame_selected[device_id] = selected

Feature Selection:   0%|          | 0/2220 [00:00<?, ?it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature gyro_y__symmetry_looking__r_0.4 is constant


Feature Selection:  22%|██▏       | 482/2220 [00:00<02:19, 12.50it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature gyro_y__mean_abs_change_quantiles__qh_0.4__ql_0.4 is constant


Feature Selection:  58%|█████▊    | 1296/2220 [00:00<00:36, 25.42it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature mag_x__symmetry_looking__r_0.35000000000000003 is constant


Feature Selection:  75%|███████▌  | 1666/2220 [00:00<00:15, 36.21it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature mag_x__large_standard_deviation__r_0.05 is constant


Feature Selection:   0%|          | 0/2220 [00:00<?, ?it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature mag_x__autocorrelation__lag_5 is constant


...

Feature Selection:   2%|▏         | 38/2220 [00:00<00:06, 312.43it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature accel_x__symmetry_looking__r_0.7000000000000001 is constant


Feature Selection:  26%|██▌       | 571/2220 [00:00<00:03, 457.33it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature accel_x__friedrich_coefficients__m_3__r_30__coeff_2 is constant


Feature Selection:  58%|█████▊    | 1296/2220 [00:00<00:01, 635.30it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature gyro_y__value_count__value_nan is constant


Feature Selection:  82%|████████▏ | 1814/2220 [00:00<00:00, 860.44it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature mag_y__large_number_of_peaks__n_3 is constant


Feature Selection: 100%|██████████| 2220/2220 [00:00<00:00, 2958.46it/s]WARNING:tsfresh.feature_selection.feature_selector:[test_feature_significance] Feature mag_x__percentage_of_reoccurring_values_to_all_values is constant



**Save to disk**

In [32]:
for device_id in data_frame_selected:
    selected = data_frame_selected[device_id]
    selected.to_pickle(features_output_path + device_id + '_selected.p')

In [33]:
for device_id in device_labels:
    print(data_frame_features[device_id].index)

Float64Index([1664.0, 1665.0, 1666.0, 1667.0, 1668.0, 1669.0, 1670.0, 1671.0,
              1672.0, 1673.0,
              ...
              2486.0, 2487.0, 2488.0, 2489.0, 2490.0, 2491.0, 2492.0, 2493.0,
              2494.0, 2495.0],
             dtype='float64', name='id', length=832)
Float64Index([ 832.0,  833.0,  834.0,  835.0,  836.0,  837.0,  838.0,  839.0,
               840.0,  841.0,
              ...
              1654.0, 1655.0, 1656.0, 1657.0, 1658.0, 1659.0, 1660.0, 1661.0,
              1662.0, 1663.0],
             dtype='float64', name='id', length=832)
Float64Index([  0.0,   1.0,   2.0,   3.0,   4.0,   5.0,   6.0,   7.0,   8.0,
                9.0,
              ...
              822.0, 823.0, 824.0, 825.0, 826.0, 827.0, 828.0, 829.0, 830.0,
              831.0],
             dtype='float64', name='id', length=832)
